# Classifying Yelp Reviews (Classifying Sentiment of Restaurant Reviews, chapter 3)

### Source: Chapter 3, Natural Language Processing with Pytorch. (2019). Delip Rao and Brian McMahan. O’Reilly: source code available on https://github.com/joosthub/PyTorchNLPBook

### PyTorch tutorial: refer to https://pytorch.org/tutorials/

## Imports

In [ ]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

## Data Vectorization classes

### The Vocabulary

In [ ]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx         # _token_to_idx: {'<UNK>':0, 'apple':1, 'banana':2, ....., 'zzz':10000}
                                                  # _idx_to_token: {0:'<UNK>', 1:'apple', 2:'banana', ....., 10000:'zzz'}

        self._idx_to_token = {idx: token          
                              for token, idx in self._token_to_idx.items()} 
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token)        

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:                   # if unk_token is defined, unknown words are handled
            return self._token_to_idx.get(token, self.unk_index)  # self.unk_index set to 0
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

### The Vectorizer

In [ ]:
class ReviewVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, review_vocab, rating_vocab):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
            rating_vocab (Vocabulary): maps class labels to integers; {'negative':0, 'positive':1}
        """
        self.review_vocab = review_vocab     # {'<UNK>':0, 'apple':1, 'banana':2, ....., 'zzz':10000}
        self.rating_vocab = rating_vocab     # {'negative':0, 'positive':1}

    def vectorize(self, review):
        """Create a collapsed one-hot vector for the review
        
        Args:
            review (str): the review 
        Returns:
            one_hot (np.ndarray): the collapsed one-hot encoding   
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)  # E.g., the one_hot vector size: 1792
        
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1

        return one_hot  # E.g., "Unfortunately, the frustration of being Dr. Go..." -> [0, 0, 1, 0, 1, ....., 0, 0]
    
    """ task 1-1
    def vectorize(self, review):
        term_frequency = np.zeros(len(self.review_vocab), dtype=np.float32)
    
        for token in review.split(" "):
            if token not in string.punctuation:
                index = self.review_vocab.lookup_token(token)
                term_frequency[index] += 1  #Accumulation if exist. this will not be affected by cutoff because the cutoff in add_token
        
        return term_frequency
    """

    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            cls: class name, i.e. ReviewVectorizer
            review_df (pandas.DataFrame): the review dataset
            cutoff (int): the parameter for frequency-based filtering
        Returns:
            an instance of the ReviewVectorizer
        """
        review_vocab = Vocabulary(add_unk=True)   # create an instance of Vocabulary class
        rating_vocab = Vocabulary(add_unk=False)
        
        # Add ratings
        for rating in sorted(set(review_df.rating)):  
            rating_vocab.add_token(rating)            # {'negative':0, 'positive':1}

        # Add top words if count > provided count
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:    # ignore punctuation tokens
                    word_counts[word] += 1
               
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)          # {'<UNK>':0, 'apple':1, 'banana':2, ....., 'zzz':10000}

        return cls(review_vocab, rating_vocab)

### The Dataset

In [ ]:
class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        Args:
            review_df (pandas.DataFrame): the dataset
            vectorizer (ReviewVectorizer): vectorizer instantiated from dataset
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)        # E.g., 3918

        self.val_df = self.review_df[self.review_df.split=='val']
        self.validation_size = len(self.val_df)     # E.g., 840

        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)          # E.g., 840

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv, frequency_cutoff):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            cls: class name, i.e. ReviewDataset
            review_csv (str): location of the dataset
        Returns:
            an instance of ReviewDataset
        """
        review_df = pd.read_csv(review_csv)
        train_review_df = review_df[review_df.split=='train']
        return cls(review_df, ReviewVectorizer.from_dataframe(train_review_df, frequency_cutoff))
    
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe 
        
        Args:
            split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        review_vector = \
            self._vectorizer.vectorize(row.review)

        rating_index = \
            self._vectorizer.rating_vocab.lookup_token(row.rating)

        return {'x_data': review_vector,           #  e.g., { 'x_data': [0, 0, 1, 0, 1, ....., 0, 0],
                'y_target': rating_index}          #          'y_target': 0  }

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size  # the floor division "//" rounds the result down to the nearest whole number
    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the cpu or gpu device location.
    """
    # drop_last: set to True to drop the last incomplete batch, if the dataset size is not divisible by the batch size. 
    # If False and the size of dataset is not divisible by the batch size, then the last batch will be smaller. (default: False)
    # need to use sampler option for balanced data: 
    # https://discuss.pytorch.org/t/balanced-sampling-between-classes-with-torchvision-dataloader/2703
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():  # name: x_data & y_target
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: ReviewClassifier

In [ ]:
class ReviewClassifier(nn.Module):
    """ a simple perceptron based classifier """
    def __init__(self, num_features, hidden_dim):
        """
        Args:
            num_features (int): the size of the input feature vector
            hidden_dim   (int): the size of hidden dimension
        """
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=hidden_dim)  # E.g., num_features: 1792; hidden_dim: 20
        self.fc2 = nn.Linear(in_features=hidden_dim, out_features=1)

        """ task 1-2
        hidden_dim2 = hidden_dim
        self.fc2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim2) 
        self.fc3 = nn.Linear(in_features=hidden_dim2, out_features=1)
        """

    def forward(self, x_in):
        """The forward pass of the classifier    
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be [batch, num_features]
        Returns:
            the resulting tensor. tensor.shape should be [batch]
        """
        intermediate = self.fc1(x_in)            # [batch, num_features] -> [batch, hidden_dim] ; E.g., [128,1792]->[128,20]
        intermediate = F.relu(intermediate)      # [batch, hidden_dim] ; E.g., [128,20]
        y_out = self.fc2(intermediate)           # [batch, hidden_dim] -> [batch, out_features] ; E.g., [128,20]->[128,1]

        """task 1-2
        intermediate = self.fc2(intermediate)
        intermediate = F.relu(intermediate)  
        y_out = self.fc3(intermediate) 
        """

        return torch.sigmoid(y_out).squeeze()    # [batch, 1] -> [batch] (e.g., [0.3, 0.1, ..., 0.5]) ; E.g., [128,1]->[128]

### Excercise 
1. Add 1 additional hidden layer

## Training Routine

### Helper functions

In [ ]:
def make_train_state():
    return {'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': []}

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (y_pred>0.5).cpu().long()
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()  # item() to get a Python number from a tensor containing a single value
    return n_correct / len(y_pred_indices) * 100

#### General utilities

In [ ]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

### Settings and some prep work

In [ ]:
args = Namespace(
    # Data and Path information
    # frequency_cutoff=25,
    frequency_cutoff=0, #task 1-3
    review_csv='data/yelp/reviews_with_splits_lite.csv',
    # Model hyper parameters
    hidden_dim=20,
    # Training hyper parameters
    batch_size=128,
    learning_rate=0.001,
    num_epochs=25,
    seed=1337,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
)
  
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

### Excercise 
1. Change the value of frequency_cutoff to 0 for using all the terms occurring in training dataset
2. Try to explore other hyperparameters.

### Initializations

In [ ]:
print("Loading dataset and creating vectorizer")
# create dataset and vectorizer
dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv, args.frequency_cutoff) 

vectorizer = dataset.get_vectorizer()

classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab), hidden_dim=args.hidden_dim)

In [ ]:
str(vectorizer.review_vocab), str(vectorizer.rating_vocab)

### Training loop

In [ ]:
classifier = classifier.to(args.device)

loss_func = nn.BCELoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

train_state = make_train_state()

epoch_bar = tqdm(desc='training routine', total=args.num_epochs)  # progress bar

dataset.set_split('train')
train_bar = tqdm(desc='split=train', total=dataset.get_num_batches(args.batch_size), leave=True)

dataset.set_split('val')
val_bar = tqdm(desc='split=val', total=dataset.get_num_batches(args.batch_size), leave=True)

try:
    for epoch_index in range(args.num_epochs):
 
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float()) # [batch, num_features]->[batch] ; E.g., [128,1792]->[128]

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float()) # ([batch],[batch])->scalar ; E.g., ([128],[128])-> 0.69
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target']) # ([batch],[batch])->scalar ; E.g., ([128],[128])-> 55.34
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            # update bar
            train_bar.set_postfix(loss=running_loss, 
                                  acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()
        
        train_state['train_loss'].append(running_loss) # train_loss for each epoch
        train_state['train_acc'].append(running_acc)   # train_acc for each epoch

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float()) # [batch, num_features]->[batch] ; E.g., [128,1792]->[128]

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, 
                                acc=running_acc, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)   # val_loss for each epoch
        train_state['val_acc'].append(running_acc)     # val_acc for each epoch

        train_bar.n = 1  # reset number of finished iterations
        val_bar.n = 1
        epoch_bar.update()
        
except KeyboardInterrupt:
    print("Exiting loop")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = train_state['train_acc']
val_acc = train_state['val_acc']
loss = train_state['train_loss']
val_loss = train_state['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# compute the loss & accuracy on the test set using the best available model

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

y_pred_list = []    # store predicted values for confusion matrix
y_target_list = []  # ground truth value

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())   # [batch, num_features]->[batch] ; E.g., [128,1792]->[128]
    
    # store predicted values and ground truth values for calculating confusion matrix
    y_pred_list.extend((y_pred>0.5).cpu().long().numpy())      # E.g., [128] -> [1, 0, 1, 0, 0, .......]
    y_target_list.extend(batch_dict['y_target'].cpu().numpy()) # E.g., [128] -> [1, 0, 0, 0, 1, .......]
    
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [ ]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

In [ ]:
rating_classes = []
for i in range(len(dataset._vectorizer.rating_vocab)):
    rating_classes.append(dataset._vectorizer.rating_vocab.lookup_index(i))
print(rating_classes)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
cm = confusion_matrix(y_target_list, y_pred_list)
cm_df = pd.DataFrame(cm.T, index=rating_classes, columns=rating_classes)
cm_df.index.name = 'Predicted'
cm_df.columns.name = 'True'
print(cm_df)

In [ ]:
print(classification_report(y_target_list, y_pred_list))

### Prediction

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)     # E.g., convert "end." to "end . "
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)  # replace special character strings with empty string
    return text

In [ ]:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    """Predict the rating of a review
    
    Args:
        review (str): the text of the review
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        decision_threshold (float): The numerical boundary which separates the rating classes
    """
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(vectorizer.vectorize(review))  # E.g., [1792]
    result = classifier(vectorized_review.view(1, -1))  # convert a vector to a matrix; E.g., [1, 1803]->[1]
    
    probability_value = result.item()
    index = 1
    if probability_value < decision_threshold:
        index = 0

    return vectorizer.rating_vocab.lookup_index(index)

In [ ]:
test_review = "this is a pretty awesome book"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

### End

# Lab Assignment #1: due on February 26, 2025

- ### Task 1: Modify the original model using following approaches. Note that this task is meant for testing the performance change of the model with sugggested changes. So try to observe the performance change (i.e., test accuracy and loss) of each suggestion, and discuss how each change affects the results of original model. You only need to modify one of the approaches below at a time, such as setting the frequencey_cutoff to 0 without using a term frequency vector or adding one additional hidden layer. However, you are allowed to explore the best combination of these approaches. 
  1. Instead of using a term presence vector (e.g., [0, 1, 0, 0, ..., 1, 0]), use a term frequuecy vector (e.g., [0, 2, 0, 0, ..., 3, 0]).
  > need to modify vectorize() function in code cell [8].
  2. Add 1 additional hidden layer between self.fc1 and self.fc2.
  > need to modify \_\_init\_\_() and forward() functions in code cell [12].
  3. Change the value of frequency_cutoff to 0 for using all the terms occurring in training dataset.
  > need to modify the value of args in code cell [20].
 
- ### Task 2: Modify the original model to have 2 output nodes (instead of one output node) in the last layer using following guidelines. Note that you don't need to strictly follow these guidelines as long as your code generates correct results. Also note that this task is meant for learning how to modify the existing model, but not for improving the performance of the model.
  1. Change the loss funcion in code cell [26]
  > loss_func = nn.BCELoss() -> loss_func = nn.CrossEntropyLoss()
  2. Change the output features size and output of the model in code cell [12].  
  > hint: just "return y_out" since softmax() is handled by nn.CrossEntropyLoss(). 
  >> return torch.sigmoid(y_out).squeeze() -> return y_out
  3. Change compute_accuracy() in code cell [16].
  > y_pred_indices = (y_pred>0.5).cpu().long() -> y_pred_indices = y_pred.max(dim=1).indices.cpu()
  4. Change all occurences of loss_func() code as below.
  > loss = loss_func(y_pred, batch_dict['y_target'].float()) -> loss = loss_func(y_pred, batch_dict['y_target'])
  5. Need to change code in code cells [29] and [36] since the new model has 2 output nodes results, instead of one node result (refer to code change in step 3).

- ### Use data\yelp\reviews_with_splits_lite.csv as a dataset in this lab assignment.
- ### Prepare a Word or PDF report summarizing your solutions for Tasks 1 and 2. For Task 1, explain the modifications made to the original code and discuss how these changes impacted the test results. For Task 2, describe the changes made to the original code and justify your solution (i.e., why your or the suggested code was used to replace the original). The report should be up to two pages in length.

- ### Submit one zip file, named __lab-assign-no1-yourname.zip__, that contains 2 Jupyter Notebook files (one for each Task; Please ensure that you display the modified code and, if necessary, disable the original code using comments (#).), input data file (include only reviews_with_splits_lite.csv file), and a report file through Turnitin on the class website. 

- ### The Jupyter Notebook file must show all output results of your solution code. So please ensure you run all the cells in the notebook file before submitting. Also, note that Turnitin does not allow you to resubmit your lab assignment file.
- ### The lab assignment handed in after the due date will not be marked. So please submit the assignment file at least one hour before the deadline date and time. Also, ensure you get a confirmation message right after you submit the assignment file via Turnitin.